#  Consignes

## Description

Ouvrir le fichier ks-projects-201801.csv, il recense environ 100 000 projets KickStarter. Intégrer les données directement avec L'API Python dans une base de données Mongo. 

Il conviendra de bien spécifier manuellement l'ID du document. Pensez aussi à bien formatter le type des données pour profiter des méthodes implémentées par Mongo. L'ensemble de données n'est pas forcément nécessaire, c'est à vous de créer votre modèle de données.

## Questions

- 1) Récupérer les 5 projets ayant reçu le plus de promesse de dons.
- 2) Compter le nombre de projets ayant atteint leur but.
- 3) Compter le nombre de projets pour chaque catégorie.
- 4) Compter le nombre de projets français ayant été instanciés avant 2016.
- 5) Récupérer les projets américains ayant demandé plus de 200 000 dollars.
- 6) Compter le nombre de projet ayant "Sport" dans leur nom

In [8]:
import pandas as pd
import pymongo
import json

In [2]:
client = pymongo.MongoClient("mongodb://192.168.1.10:27017/")
database = client['exercices']
collection = database['kickstarter']

In [3]:
df_ks = pd.read_csv("./data/ks-projects-201801-sample.csv")
df_ks.head()

/root/.local/share/virtualenvs/code-AFz48Hjb/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (6,8,10,12) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd pledged,usd_pledged_real
0,872782264,"Scott Cooper's Solo CD ""A Leg Trick"" (Canceled)",Rock,Music,USD,2011-09-16,2000,2011-08-17 06:31:31,1145,canceled,24,US,1145,1145.000000
1,1326492673,Ohceola jewelry,Fashion,Fashion,USD,2012-08-22,18000,2012-07-23 20:46:48,1851,failed,28,US,1851,1851.000000
2,1688410639,Sluff Off & Harald: Two latest EGGs are Classi...,Tabletop Games,Games,USD,2016-07-19,2000,2016-07-01 21:55:54,7534,successful,254,US,3796,7534.000000
3,156812982,SketchPlanner: Create and Plan- all in one bea...,Art Books,Publishing,USD,2017-09-27,13000,2017-08-28 15:47:02,16298,successful,367,US,2670,16298.000000
4,1835968190,Proven sales with custom motorcycle accessories,Sculpture,Art,CAD,2016-02-24,5000,2016-01-25 17:37:10,1,failed,1,CA,0.708148,0.738225


Ce warning intervient lorsque pandas n'arrive pas à inférer le type de données. Il est sympa il précise les colones 6,8,10,12. 

In [4]:
df_ks.columns[[6,8,10,12]]

Index(['goal', 'pledged', 'backers', 'usd pledged'], dtype='object')

## Question 0

### Netoyer les données

In [5]:
df_ks.isna().sum()

ID                     0
name                   2
category               0
main_category          0
currency               0
deadline               0
goal                   0
launched               0
pledged                0
state                  0
backers                0
country                0
usd pledged         1482
usd_pledged_real       0
dtype: int64

In [7]:
df_ks = df_ks.dropna()
df_ks.isna().sum()

ID                  0
name                0
category            0
main_category       0
currency            0
deadline            0
goal                0
launched            0
pledged             0
state               0
backers             0
country             0
usd pledged         0
usd_pledged_real    0
dtype: int64

### Importer les données

In [23]:
collection.insert_many(df_ks.to_dict('records'))

In [36]:
cursor = collection.find()

In [66]:
next(cursor)

{'_id': ObjectId('61ab91d2d7efb6f56f78945d'),
 'ID': 1390262683,
 'name': 'Candy Coburn - "THE GOOD LIFE" New Soul Rockin\' Studio Album',
 'category': 'Music',
 'main_category': 'Music',
 'currency': 'USD',
 'deadline': '2016-06-08',
 'goal': 15000.0,
 'launched': '2016-05-10 12:43:24',
 'pledged': 15530.0,
 'state': 'successful',
 'backers': 151,
 'country': 'US',
 'usd pledged': 15530.0,
 'usd_pledged_real': 15530.0}

## Question 1  

In [77]:
cur = collection.find().sort([("usd_pledged_real", -1)])
print(next(cur))
print()
print(next(cur))
print()
print(next(cur))
print()
print(next(cur))
print()
print(next(cur))

{'_id': ObjectId('61ab91d2d7efb6f56f79bc5e'), 'ID': 342886736, 'name': "COOLEST COOLER: 21st Century Cooler that's Actually Cooler", 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': '2014-08-30', 'goal': '50000.0', 'launched': '2014-07-08 10:14:37', 'pledged': '13285226.36', 'state': 'successful', 'backers': '62642', 'country': 'US', 'usd pledged': '13285226.36', 'usd_pledged_real': 13285226.36}

{'_id': ObjectId('61ab91d2d7efb6f56f7a1688'), 'ID': 2103598555, 'name': 'Pebble 2, Time 2 + All-New Pebble Core', 'category': 'Product Design', 'main_category': 'Design', 'currency': 'USD', 'deadline': '2016-06-30', 'goal': '1000000.0', 'launched': '2016-05-24 15:49:52', 'pledged': '12779843.49', 'state': 'successful', 'backers': '66673', 'country': 'US', 'usd pledged': '12779843.49', 'usd_pledged_real': 12779843.49}

{'_id': ObjectId('61ab91d3d7efb6f56f7a4b1f'), 'ID': 1033978702, 'name': 'OUYA: A New Kind of Video Game Console', 'category': 'Gaming Hardw

## Question 2

## Question 3

## Question 4

## Question 5

## Question 6 